# Automatic speech recognition with a pipeline

Otomatik Konuşma Tanıma (ASR), konuşma ses kaydının metne dönüştürülmesini içeren bir görevdir. Bu görevin, videolar için altyazı oluşturmaktan Siri ve Alexa gibi sanal asistanlar için sesli komutları etkinleştirmeye kadar çok sayıda pratik uygulaması vardır.

Bu bölümde, daha önce olduğu gibi aynı MINDS-14 veri kümesini kullanarak fatura ödeme hakkında soru soran bir kişinin ses kaydını yazıya dökmek için otomatik konuşma tanıma işlem hattını kullanacağız.

Başlamak için, veri kümesini yükleyin ve henüz yapmadıysanız, bir işlem hattı ile ses sınıflandırma bölümünde açıklandığı gibi 16kHz'e yükseltin.

Bir ses kaydını yazıya dökmek için Transformers'daki otomatik konuşma tanıma pipeline'ını kullanabiliriz.

In [1]:
from datasets import load_dataset, Audio

minds = load_dataset(
    "PolyAI/minds14", name="en-AU", 
    split="train", trust_remote_code=True,
)

minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 654
})

In [2]:
from transformers import pipeline

asr = pipeline("automatic-speech-recognition")

2024-08-08 07:27:58.675165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 07:27:58.675337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 07:27:58.915200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [3]:
example = minds[0]
asr(example["audio"]["array"])

{'text': 'I WOULD LIKE TO PAY MY ELECTRICITY BILL USING MY CAD CAN YOU PLEASE ASSIST'}

In [4]:
example["english_transcription"]

'I would like to pay my electricity bill using my card can you please assist'

Model, sesin transkripsiyonunda oldukça iyi bir iş çıkarmış gibi görünüyor! Orijinal transkripsiyona kıyasla yalnızca bir kelimeyi ("card") yanlış yazmış ki bu da konuşmacının "r" harfinin genellikle sessiz olduğu bir Avustralya aksanına sahip olduğu düşünüldüğünde oldukça iyi.

Varsayılan olarak, bu işlem hattı İngilizce dili için otomatik konuşma tanıma için eğitilmiş bir model kullanır, bu da bu örnek için uygundur. MINDS-14'ün diğer alt kümelerini farklı bir dilde yazıya dökmeyi denemek isterseniz, Hub'da önceden eğitilmiş bir ASR modeli bulabilirsiniz. Modeller listesini önce göreve, ardından dile göre filtreleyebilirsiniz. Beğendiğiniz modeli bulduğunuzda, modelin adını boru hattına model bağımsız değişkeni olarak aktarın.

Bunu MINDS-14'ün Almanca bölümü için deneyelim. "de-DE" alt kümesini yükleyin:

In [5]:
minds = load_dataset("PolyAI/minds14", name="de-DE", split="train")
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 611
})

In [6]:
example = minds[0]
example["transcription"]

'ich möchte gerne Geld auf mein Konto einzahlen'

In [7]:
asr = pipeline("automatic-speech-recognition", model="maxidl/wav2vec2-large-xlsr-german")
asr(example["audio"]["array"])

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at maxidl/wav2vec2-large-xlsr-german were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertF

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

{'text': 'ich möchte gerne geld auf mein konto einzallen'}

Kendi görevinizi çözmeye çalışırken, bu ünitede gösterdiğimiz gibi basit bir boru hattı ile başlamak, çeşitli avantajlar sunan değerli bir araçtır:

- Görevinizi gerçekten iyi çözen önceden eğitilmiş bir model mevcut olabilir, bu da size bolca zaman kazandırır
- pipeline() sizin için tüm ön/son işlemlerle ilgilenir, böylece verileri bir model için doğru biçime getirme konusunda endişelenmenize gerek kalmaz
- Sonuç ideal olmasa da, bu size ileride ince ayar yapmak için hızlı bir temel sağlar
- Özel verileriniz üzerinde bir model üzerinde ince ayar yapıp Hub'da paylaştığınızda, tüm topluluk pipeline() yöntemi aracılığıyla bunu hızlı ve zahmetsiz bir şekilde kullanabilecek ve yapay zeka daha erişilebilir hale gelecektir.